In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
import pickle

In [3]:
df = pd.read_csv('/content/drive/MyDrive/집교 2_Team P/user-track-listen_count_filtered5.csv')

In [4]:
# Pickle 파일 읽기
with open('/content/drive/MyDrive/집교 2_Team P/lyrics_Embedding/all_embeddings_full.pkl', 'rb') as file:
    data = pickle.load(file)

# DataFrame으로 변환
df_embedding = pd.DataFrame(data, columns=['embedding', 'track_id'])

# track_id를 정수로 변환 (필요하다면)
df_embedding['track_id'] = df_embedding['track_id'].astype(int)

# 'embedding' 열을 768차원의 각 차원으로 나누기
# df_embedding[['embedding_{}'.format(i) for i in range(768)]] = pd.DataFrame(df_embedding['embedding'].tolist(), index=df_embedding.index)

# 'embedding' 열 삭제
# df_embedding = df_embedding.drop(['embedding'], axis=1)

# DataFrame 확인
print(df_embedding.head())


                                           embedding  track_id
0  [0.012072664, 0.17292306, 0.0061238254, 0.0707...         2
1  [-0.17554894, 0.24209566, 0.4195969, -0.185033...         8
2  [-0.096951924, 0.0034472912, 0.005701333, 0.01...      1524
3  [-0.21775067, 0.244962, 0.24090661, 0.1647732,...      1785
4  [-0.069424234, -0.016805744, 0.21406727, -0.27...      1787


In [5]:
print(df.shape)
df = pd.merge(df, df_embedding, on='track_id', how='inner')
df.shape

(4645010, 4)


(4644051, 5)

In [6]:
from sklearn.preprocessing import LabelEncoder
user_encoder = LabelEncoder()
track_encoder = LabelEncoder()
lyrics_encoder = LabelEncoder()
df['user_id'] = user_encoder.fit_transform(df['user_id'])
df['track_id'] = track_encoder.fit_transform(df['track_id'])
df_embedding['track_id'] = lyrics_encoder.fit_transform(df_embedding['track_id'])

In [7]:
lyrics_dict = dict(zip(df_embedding['track_id'], df_embedding['embedding']))

In [8]:
# !pip install torch torchvision -U

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from tqdm import tqdm

# 데이터 불러오기 (예시: CSV 파일)
# Label Encoding
# PyTorch DataLoader에 맞게 데이터 변환
def df_to_tensor(dataset):
    users = torch.tensor(dataset['user_id'].values, dtype=torch.int)
    items = torch.tensor(dataset['track_id'].values, dtype=torch.int)
    ratings = torch.tensor(dataset['listen_count_bin'].values, dtype=torch.float)
    lyrics_embeddings = torch.tensor(np.vstack(dataset['embedding'].values), dtype=torch.float)
    return users, items, ratings, lyrics_embeddings

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_users, train_items, train_ratings, train_lyrics_embeddings = df_to_tensor(train_df)
test_users, test_items, test_ratings,test_lyrics_embeddings = df_to_tensor(test_df)

train_data = TensorDataset(train_users, train_items, train_ratings,train_lyrics_embeddings)
test_data = TensorDataset(test_users, test_items, test_ratings,test_lyrics_embeddings)

train_loader = DataLoader(train_data, batch_size=256, shuffle=True)
test_loader = DataLoader(test_data, batch_size=256, shuffle=False)
num_users = (df['user_id'].nunique())
num_items = (df['track_id'].nunique())
print(num_users)
print(num_items)

23761
28309


In [9]:
# NCF 모델 정의
class NCF_embedding(nn.Module):
    def __init__(self, num_users, num_items, embedding_size):
        super(NCF_embedding, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_size)
        self.item_embedding = nn.Embedding(num_items, embedding_size)
        self.lyrics_embedding = nn.Linear(768, embedding_size)
        self.fc_layers = nn.Sequential(
            nn.Linear(embedding_size * 3, embedding_size),
            nn.ReLU(),
            nn.Linear(embedding_size, int(embedding_size/2)),
            nn.ReLU(),
            nn.Linear(int(embedding_size/2), int(embedding_size/4)),
            nn.ReLU(),
            nn.Linear(int(embedding_size/4), int(embedding_size/8)),
            nn.ReLU(),
            nn.Linear(int(embedding_size/8), 1)
        )

    def forward(self, user, item, lyrics_embedding):
        user_embedding = self.user_embedding(user)
        item_embedding = self.item_embedding(item)
        lyrics_embedding = self.lyrics_embedding(lyrics_embedding.reshape(lyrics_embedding.shape[0],768))
        x = torch.cat((user_embedding, item_embedding,lyrics_embedding), dim=1)
        x = self.fc_layers(x)
        return x

# CUDA 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
def train_embedding(embedding,n_epoch):
  NCF 모델 정의 및 GPU로 이동
  model = NCF_embedding(num_users=num_users, num_items=num_items, embedding_size=embedding)
  model.to(device)
  criterion = nn.MSELoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001)
  # tqdm을 사용하여 학습 및 테스트 진행 상황 확인
  num_epochs = n_epoch
  for epoch in range(num_epochs):
      model.train()
      total_loss = 0
      for user, item, rating,lyrics_embedding in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
          optimizer.zero_grad()
          user, item, rating,lyrics_embedding = user.to(device), item.to(device), rating.to(device),lyrics_embedding.to(device)  # GPU로 이동
          output = model(user, item,lyrics_embedding.float())
          loss = criterion(output, rating.unsqueeze(1))
          loss.backward()
          optimizer.step()
          total_loss += loss.item()

      avg_loss = total_loss / len(train_loader)
      print(f'Epoch {epoch+1}/{num_epochs}, Avg. Loss: {avg_loss:.4f}')

      # 각 에폭이 끝날 때마다 테스트 데이터에 대한 예측 수행
      model.eval()
      all_predictions = []
      with torch.no_grad():
          for user, item, _,lyrics_embedding in tqdm(test_loader, desc=f'Testing Epoch {epoch+1}'):
              user, item,lyrics_embedding = user.to(device), item.to(device),lyrics_embedding.to(device)  # GPU로 이동
              output = model(user, item,lyrics_embedding)
              all_predictions.append(output)

      # RMSE 계산
      predictions = torch.cat(all_predictions).squeeze().cpu().numpy()  # CPU로 이동 후 numpy로 변환
      rmse = np.sqrt(mean_squared_error(test_df['listen_count_bin'].values, predictions))
      print(f'Epoch {epoch+1}/{num_epochs}, RMSE on test set: {rmse}')


Using device: cuda


In [12]:
train_embedding(64,23)

Epoch 1/23: 100%|██████████| 14513/14513 [01:40<00:00, 144.00it/s]


Epoch 1/23, Avg. Loss: 1.4111


Testing Epoch 1: 100%|██████████| 3629/3629 [00:15<00:00, 229.45it/s]


Epoch 1/23, RMSE on test set: 1.154973089505285


Epoch 2/23: 100%|██████████| 14513/14513 [01:44<00:00, 138.78it/s]


...

Epoch 22/23, RMSE on test set: 1.1350342434765748


Epoch 23/23: 100%|██████████| 14513/14513 [01:44<00:00, 138.41it/s]


Epoch 23/23, Avg. Loss: 1.0232


Testing Epoch 23: 100%|██████████| 3629/3629 [00:15<00:00, 230.29it/s]

Epoch 23/23, RMSE on test set: 1.1336342040072844


In [ ]:
train_embedding(256,30)

Epoch 1/30: 100%|██████████| 14521/14521 [01:57<00:00, 123.97it/s]


Epoch 1/30, Avg. Loss: 1.3946


Testing Epoch 1: 100%|██████████| 3631/3631 [00:18<00:00, 196.90it/s]


Epoch 1/30, RMSE on test set: 1.1583708911674413


Epoch 2/30: 100%|██████████| 14521/14521 [01:59<00:00, 121.64it/s]


...

Epoch 14/30, Avg. Loss: 1.0008


Testing Epoch 14: 100%|██████████| 3631/3631 [00:17<00:00, 206.68it/s]


Epoch 14/30, RMSE on test set: 1.1336277103503922


Epoch 15/30:   6%|▌         | 853/14521 [00:07<01:54, 119.24it/s]


KeyboardInterrupt: ignored

In [ ]:
train_embedding(512,15)

Epoch 1/15: 100%|██████████| 14521/14521 [02:13<00:00, 108.80it/s]


Epoch 1/15, Avg. Loss: 1.3878


Testing Epoch 1: 100%|██████████| 3631/3631 [00:17<00:00, 202.98it/s]


Epoch 1/15, RMSE on test set: 1.156338076116727


Epoch 2/15: 100%|██████████| 14521/14521 [02:19<00:00, 104.36it/s]


...

Epoch 14/15, RMSE on test set: 1.1244599452272597


Epoch 15/15: 100%|██████████| 14521/14521 [02:18<00:00, 104.60it/s]


Epoch 15/15, Avg. Loss: 0.9918


Testing Epoch 15: 100%|██████████| 3631/3631 [00:17<00:00, 202.50it/s]

Epoch 15/15, RMSE on test set: 1.121546484576387


In [ ]:
train_embedding(768,30)

Epoch 1/30: 100%|██████████| 14513/14513 [02:42<00:00, 89.29it/s]


Epoch 1/30, Avg. Loss: 1.3863


Testing Epoch 1: 100%|██████████| 3629/3629 [00:18<00:00, 196.73it/s]


Epoch 1/30, RMSE on test set: 1.154910520886283


Epoch 2/30: 100%|██████████| 14513/14513 [02:45<00:00, 87.48it/s]


...

Epoch 20/30, Avg. Loss: 0.8125


Testing Epoch 20: 100%|██████████| 3629/3629 [00:18<00:00, 197.67it/s]


Epoch 20/30, RMSE on test set: 1.1271186920101732


Epoch 21/30:  61%|██████    | 8841/14513 [01:48<01:09, 81.67it/s]


KeyboardInterrupt: ignored